# Daten gruppieren, aggregieren und pivotieren


In [1]:
# Modulimporte
import pandas as pd
import numpy as np
import seaborn as sns


# Daten zusammenfassen - Aggregation

Aggregationsfunktionen kennen wir schon viele: `max`, `min`, `sum`, `count`, `mean`, `average`, ...
Aggregation bedeutet so etwas wie "Verdichtung" oder Zusammenfassung. 
Dabei werden viele Werte genommen und am Ende ein Wert daraus gebildet oder gezogen.
Beim Maximum ist es etwa der größte Wert aus einer Datenreihe.
Beim Mittelwert ist es die Summe aller Werte einer Datenreihe geteilt durch die Anzahl der Werte in der Datenreihe.

<br>Aggregationsfunktionen können auch auf DataFrames angewandt werden. Dann werden die Funktionen spaltenweise ausgeführt.


In [2]:
df = sns.load_dataset("planets")
df.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


# Liste einiger Aggregationsfunktionen
df.min()
df.max()
df.sum()
df.prod()
df.mean()
df.median()

df.count()
df.unique()

In [3]:
# Anwendung von Aggregation auf den ganzen Datensatz
# Gibt größten Wert jeder Spalte zurück
df.max()

method            Transit Timing Variations
number                                    7
orbital_period                     730000.0
mass                                   25.0
distance                             8500.0
year                                   2014
dtype: object

In [4]:
# Ist NICHT eine einzige Zeile!
df[df["orbital_period"] == 730000]

,method,number,orbital_period,mass,distance,year
937,Imaging,1,730000.0,NaN,NaN,2006


In [5]:
# Anwendung von Aggregation auf eine Spalte
df["mass"].max()

np.float64(25.0)

In [6]:
# Herausfinden, um welche Zeile es sich handelt:
df['mass'].idxmax()

321

In [7]:
# Nur die Zeile mit dem Maximum holen:
max_mass = df['mass'].idxmax()
df.iloc[max_mass, :]

method            Radial Velocity
number                          1
orbital_period             2371.0
mass                         25.0
distance                    37.05
year                         2008
Name: 321, dtype: object

In [8]:
# Anwendung von Aggregation auf mehrere Spalten
df[["mass", "distance"]].max()

mass          25.0
distance    8500.0
dtype: float64

In [9]:
# Quizfrage: Warum wird hier idmax nicht funktionieren?
# Und wie löst man das dann?

In [10]:
mass_max = df['mass'].idxmax()
distance_max = df['distance'].idxmax()

In [11]:
df.iloc[[mass_max, distance_max]]

,method,number,orbital_period,mass,distance,year
321,Radial Velocity,1,2371.0,25.0,37.05,2008
951,Transit,1,4.2,NaN,8500.00,2006


In [12]:
# Anwendung von Aggregation auf mehrere Spalten und 
# begrenzte Zeilen (es werden die Maxima aus den ersten 
# 6 Zeilen gebildet:
df.loc[:5, ["mass", "distance", "year"]].max()

mass          19.40
distance     119.47
year        2011.00
dtype: float64

In [13]:
df.loc[:5]

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009
5,Radial Velocity,1,185.840,4.80,76.39,2008


In [14]:
# Anwendung von mehreren Aggregationen (auf den kompletten df)
# --> agg()
df.agg(["max", "min"])

,method,number,orbital_period,mass,distance,year
max,Transit Timing Variations,7,730000.000000,25.0000,8500.00,2014
min,Astrometry,1,0.090706,0.0036,1.35,1989


In [15]:
# Anwendung selbst definierter Funktionen mit .agg()
def aggfunc(x: pd.Series) -> int:
    # Entfernt alle NaN Werte der Serie
    x.dropna(inplace=True)
    return x.size

In [16]:
# Ohne Entfernung der NaN
df["mass"].size

1035

In [17]:
# Mit der eigenen Funktion werden alle
# NaN entfernt vor dem zählen
df["mass"].agg(aggfunc)

513

In [18]:
# Auch Lambda-Funktionen mit .agg() möglich
df[["mass", "distance"]].agg(lambda x: np.sum(x))

mass          1353.37638
distance    213367.98000
dtype: float64

##### Übungsaufgabe

Errechne für den kompletten Datensatz "taxis" aus dem seaborn package den prozentualen Anteil des Trinkgelds (`tip`) am Gesamtpreis der Fahrt (`total`) und lasse dir im Anschluss das Minimum, Maximum und den Mittelwert dieses neuen Wertes ausgeben.

In [53]:
#######################################
# Load Taxis DataSet & save it in Df: #
#######################################
taxis_df = sns.load_dataset('taxis')

##################################################
# calculate Tip percentage share for Total:      #
# by creating 'perc_tip' Column, than calculate: #
##################################################
taxis_df['perc_tip'] = (taxis_df['tip'] / taxis_df['total']) * 100

###############
# ALTERNATIV: #
###############
# taxis_df['perc_tip'].agg(['min', 'max', 'mean'])

##################################################
# calculate min / max / mean for percentage Tip: #
##################################################
perc_tip_min = taxis_df['perc_tip'].min()
perc_tip_max = taxis_df['perc_tip'].max()
perc_tip_mean = taxis_df['perc_tip'].mean()

############
# RESULTS: #
############
print(f'Min Tip Percentage = {perc_tip_min:.2f} %')
print(f'Max Tip Percentage = {perc_tip_max:.2f} %')
print(f'Mean Tip Percentage = {perc_tip_mean:.2f} %')

##########################
# create new Results Df: #
##########################
results_df = pd.DataFrame(
    {'Stats': ['Min %', 'Max %', 'Mean %'],
    'Tip Percentage Share': [perc_tip_min, perc_tip_max, perc_tip_mean]},
    index=['0️⃣', '1️⃣', '2️⃣']
)

results_df

Min Tip Percentage = 0.00 %
Max Tip Percentage = 38.25 %
Mean Tip Percentage = 10.03 %


,Stats,Tip Percentage Share
0️⃣,Min %,0.000000
1️⃣,Max %,38.251366
2️⃣,Mean %,10.031654


# Gruppieren

Wir können auch Aggregationen für gewisse Kategorien in unseren Daten durchführen.
Dabei muss nach einer Spalte gruppiert werden, die kategorische Werte beinhaltet (etwa Pinguinarten)
und danach die gewünschte Aggregationsfunktion auf diese Gruppierung angewandt werden (etwa Summe).
Auf diese Weise wird für jedes Label (die drei Pinguinarten etwa) jeweils eine Summe gebildet.

In [19]:
# Gruppieren nach einer Spalte
df.groupby("method")

Mit einem einfachen groupby sagt uns Python nur, wo das Objekt gespeichert ist. Um gruppierte Informationen daraus zu erhalten, müssen wir Python noch sagen, wie die Gruppen aggregiert werden sollen (etwa: das Minimum ermitteln oder die Summe bilden)

In [20]:
# Aggregation der entstandenen Gruppen
# z.B. Minimum jeder Gruppe
df.groupby("method").min()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1,246.360000,NaN,14.98,2010
Eclipse Timing Variations,1,1916.250000,4.2000,130.72,2008
Imaging,1,4639.150000,NaN,7.69,2004
Microlensing,1,1825.000000,NaN,1760.00,2004
Orbital Brightness Modulation,1,0.240104,NaN,1180.00,2011
Pulsar Timing,1,0.090706,NaN,1200.00,1992
Pulsation Timing Variations,1,1170.000000,NaN,NaN,2007
Radial Velocity,1,0.736540,0.0036,1.35,1989
Transit,1,0.355000,1.4700,38.00,2002


In [21]:
# Man kann ein GroupBy Objekt speichern und indizieren
grps = df.groupby("method")
grps.get_group("Transit")

,method,number,orbital_period,mass,distance,year
91,Transit,1,1.508956,NaN,NaN,2008
92,Transit,1,1.742994,NaN,200.0,2008
93,Transit,1,4.256800,NaN,680.0,2008
94,Transit,1,9.202050,NaN,NaN,2008
95,Transit,1,4.037896,NaN,NaN,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.0,2006
1031,Transit,1,2.615864,NaN,148.0,2007
1032,Transit,1,3.191524,NaN,174.0,2007
1033,Transit,1,4.125083,NaN,293.0,2008


In [22]:
# Loopen durch die einzelnen Gruppen auch möglich
for name, group in df.groupby("method"):
    print(name + '\n')
    print(group)
    print()  # macht Newline

Astrometry

         method  number  orbital_period  mass  distance  year
113  Astrometry       1          246.36   NaN     20.77  2013
537  Astrometry       1         1016.00   NaN     14.98  2010

Eclipse Timing Variations

                       method  number  orbital_period  mass  distance  year
32  Eclipse Timing Variations       1        10220.00  6.05       NaN  2009
37  Eclipse Timing Variations       2         5767.00   NaN    130.72  2008
38  Eclipse Timing Variations       2         3321.00   NaN    130.72  2008
39  Eclipse Timing Variations       2         5573.55   NaN    500.00  2010
40  Eclipse Timing Variations       2         2883.50   NaN    500.00  2010
41  Eclipse Timing Variations       1         2900.00   NaN       NaN  2011
42  Eclipse Timing Variations       1         4343.50  4.20       NaN  2012
43  Eclipse Timing Variations       2         5840.00   NaN       NaN  2011
44  Eclipse Timing Variations       2         1916.25   NaN       NaN  2011

Imaging

    

In [23]:
# Am Ende sind die Gruppen einfach nur Tupel aus Label (String) und Teil-Dataframe:
for name, group in df.groupby("method"):
    print('Label:', type(name))
    print('Gruppe:', type(group))

Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>
Label: <class 'str'>
Gruppe: <class 'pandas.core.frame.DataFrame'>


In [24]:
# Gruppieren nach einer Spalte ("method"), Ausgabe
# nur einer Spalte ("distance"), davon jedoch nur 
# die Maximalwerte und sortiert
df.groupby("method")["distance"].max().sort_values(ascending=False)

method
Transit                          8500.00
Microlensing                     7720.00
Transit Timing Variations        2119.00
Pulsar Timing                    1200.00
Orbital Brightness Modulation    1180.00
Eclipse Timing Variations         500.00
Radial Velocity                   354.00
Imaging                           165.00
Astrometry                         20.77
Pulsation Timing Variations          NaN
Name: distance, dtype: float64

In [25]:
# Oder unter Verwendung unserer Variable
grps = df.groupby("method")
grps["distance"].max().sort_values()

method
Astrometry                         20.77
Imaging                           165.00
Radial Velocity                   354.00
Eclipse Timing Variations         500.00
Orbital Brightness Modulation    1180.00
Pulsar Timing                    1200.00
Transit Timing Variations        2119.00
Microlensing                     7720.00
Transit                          8500.00
Pulsation Timing Variations          NaN
Name: distance, dtype: float64

In [26]:
# Mehrere Aggregationsfunktionen pro Gruppe
# Gruppiert nach "number", nur die Infos aus "distance"
df.groupby("number")["distance"].agg(["mean", "min", "max"])

,mean,min,max
number,,,
1,291.743606,1.35,8500.00
2,257.691646,8.82,4080.00
3,125.906364,6.06,1400.00
4,15.932000,4.70,39.94
5,243.382500,12.53,368.00
6,168.005000,6.80,613.00
7,780.000000,780.00,780.00


In [27]:
# Mehrere Aggregationsfunktionen für mehrere Spalten
df.groupby("number")[["mass", "orbital_period"]] \
    .agg(["mean", "min", "max"])

mass                  orbital_period                        
            mean      min     max           mean       min           max
number                                                                  
1       3.329634  0.00360  25.000    2641.632828  0.090706  730000.00000
2       2.229547  0.00870  21.420     630.923360  0.240104    9017.80000
3       0.916872  0.00755   8.060     478.818182  0.453285   14002.00000
4       0.986492  0.00600   4.132    9564.494437  1.937780  170000.00000
5       1.166750  0.16500   3.530     199.819305  0.736540    4909.00000
6       0.038556  0.00800   0.205     170.442667  4.312300    2248.00000
7            NaN      NaN     NaN     119.217898  7.008151     331.60059

In [28]:
# Verschiedene Aggregationsfunktionen für konkrete Spalten
df.groupby("number") \
    .agg({"mass": "mean",
          "orbital_period": "max"})

,mass,orbital_period
number,,
1,3.329634,730000.00000
2,2.229547,9017.80000
3,0.916872,14002.00000
4,0.986492,170000.00000
5,1.166750,4909.00000
6,0.038556,2248.00000
7,NaN,331.60059


In [29]:
# Mit Benennung der neuen Spalten:
df.groupby("number") \
    .agg(mean_mass=('mass', 'mean'),
         max_orbital=('orbital_period', 'max')
)

,mean_mass,max_orbital
number,,
1,3.329634,730000.00000
2,2.229547,9017.80000
3,0.916872,14002.00000
4,0.986492,170000.00000
5,1.166750,4909.00000
6,0.038556,2248.00000
7,NaN,331.60059


In [30]:
# Gleichzeitig nach mehreren Spalten gruppieren
df.groupby(["number", "method"]) \
    .agg({"mass": "mean",
          "orbital_period": "max"})

mass  orbital_period
number method                                                 
1      Astrometry                          NaN     1016.000000
       Eclipse Timing Variations      5.125000    10220.000000
       Imaging                             NaN   730000.000000
       Microlensing                        NaN     3600.000000
       Orbital Brightness Modulation       NaN        1.544929
       Pulsar Timing                       NaN    36525.000000
       Pulsation Timing Variations         NaN     1170.000000
       Radial Velocity                3.323939    17337.500000
       Transit                        1.470000      289.862300
2      Eclipse Timing Variations           NaN     5840.000000
       Microlensing                        NaN     5100.000000
       Orbital Brightness Modulation       NaN        0.342887
       Radial Velocity                2.229547     9017.800000
       Transit                             NaN      303.137000
       Transit Timing Variations           NaN      160.000000
3      Pulsar Timing                       NaN       98.211400
       Radial Velocity                0.916872    14002.000000
       Transit                             NaN      143.343940
       Transit Timing Variations           NaN             NaN
4      Imaging                             NaN   170000.000000
       Radial Velocity                0.986492     4205.800000
       Transit                             NaN       54.320310
5      Radial Velocity                1.166750     4909.000000
       Transit                             NaN      267.291000
6      Radial Velocity                0.038556     2248.000000
       Transit                             NaN      118.380700
7      Transit                             NaN      331.600590

In [31]:
# Verschiedene Aggregationsfunktionen für verschiedene Spalten
df.groupby("number") \
    .agg({"mass": ["mean", "median"],
          "orbital_period": ["max", "min"]})

mass         orbital_period          
            mean  median            max       min
number                                           
1       3.329634  1.7900   730000.00000  0.090706
2       2.229547  0.9900     9017.80000  0.240104
3       0.916872  0.0580    14002.00000  0.453285
4       0.986492  0.6876   170000.00000  1.937780
5       1.166750  0.4860     4909.00000  0.736540
6       0.038556  0.0194     2248.00000  4.312300
7            NaN     NaN      331.60059  7.008151

In [58]:
taxis_df.dropna(inplace=True)

# groupby() mit numerischen Var problematisch:
taxis_df.groupby('tip').agg(['min', 'max'])

pickup                                 dropoff  \
                      min                 max                 min   
tip                                                                 
0.00  2019-02-28 23:29:03 2019-03-31 23:43:45 2019-02-28 23:32:35   
0.01  2019-03-03 21:15:32 2019-03-28 14:34:59 2019-03-03 21:40:15   
0.02  2019-03-02 11:12:13 2019-03-26 15:06:47 2019-03-02 11:25:25   
0.06  2019-03-02 12:25:56 2019-03-02 12:25:56 2019-03-02 12:45:44   
0.08  2019-03-09 03:33:20 2019-03-09 03:33:20 2019-03-09 03:46:08   
...                   ...                 ...                 ...   
16.39 2019-03-27 18:08:31 2019-03-27 18:08:31 2019-03-27 19:01:45   
16.59 2019-03-10 20:27:25 2019-03-10 20:27:25 2019-03-10 20:28:33   
18.30 2019-03-15 15:10:03 2019-03-15 15:10:03 2019-03-15 16:25:25   
20.80 2019-03-28 15:58:52 2019-03-28 15:58:52 2019-03-28 15:59:25   
23.19 2019-03-29 17:32:20 2019-03-29 17:32:20 2019-03-29 18:53:52   

                          passengers     distance          fare          ...  \
                      max        min max      min    max    min     max  ...   
tip                                                                      ...   
0.00  2019-04-01 00:13:58          0   6     0.00  36.70   1.00  150.00  ...   
0.01  2019-03-28 14:38:34          1   1     0.00   2.91   2.50   16.00  ...   
0.02  2019-03-26 15:59:52          1   1     2.37  18.41  11.00   52.00  ...   
0.06  2019-03-02 12:45:44          5   5    10.18  10.18  29.50   29.50  ...   
0.08  2019-03-09 03:46:08          1   1     4.12   4.12  15.00   15.00  ...   
...                   ...        ...  ..      ...    ...    ...     ...  ...   
16.39 2019-03-27 19:01:45          1   1    16.77  16.77  52.00   52.00  ...   
16.59 2019-03-10 20:28:33          1   1     0.00   0.00  52.00   52.00  ...   
18.30 2019-03-15 16:25:25          1   1    16.30  16.30  52.00   52.00  ...   
20.80 2019-03-28 15:59:25          1   1     1.80   1.80  69.06   69.06  ...   
23.19 2019-03-29 18:53:52          1   1    26.92  26.92  75.50   75.50  ...   

                   pickup_zone                           \
                           min                      max   
tip                                                       
0.00   Allerton/Pelham Gardens           Yorkville West   
0.01         East Harlem North  Queensbridge/Ravenswood   
0.02               Boerum Hill         Manhattan Valley   
0.06         LaGuardia Airport        LaGuardia Airport   
0.08           Lower East Side          Lower East Side   
...                        ...                      ...   
16.39              JFK Airport              JFK Airport   
16.59      Lincoln Square East      Lincoln Square East   
18.30              Murray Hill              Murray Hill   
20.80              JFK Airport              JFK Airport   
23.19              JFK Airport              JFK Airport   

                  dropoff_zone                        pickup_borough  \
                           min                    max            min   
tip                                                                    
0.00   Allerton/Pelham Gardens         Yorkville West          Bronx   
0.01                    Corona  Upper East Side South       Brooklyn   
0.02         East Harlem North                   SoHo       Brooklyn   
0.06              Midtown East           Midtown East         Queens   
0.08         Battery Park City      Battery Park City      Manhattan   
...                        ...                    ...            ...   
16.39             East Village           East Village         Queens   
16.59      Lincoln Square East    Lincoln Square East      Manhattan   
18.30              JFK Airport            JFK Airport      Manhattan   
20.80              JFK Airport            JFK Airport         Queens   
23.19              Cobble Hill            Cobble Hill         Queens   

                 dropoff_borough              perc_tip             
             max           

In [64]:
taxis_df.groupby(['pickup_borough', 'dropoff_borough'])\
    .agg(
    fare_mean=('fare', 'mean'),
    distance_mean=('distance', 'mean')
)

fare_mean  distance_mean
pickup_borough dropoff_borough                          
Bronx          Bronx            14.539091       3.677273
               Brooklyn         54.062500      13.492500
               Manhattan        29.698000       8.562800
               Queens           40.157500      14.030000
Brooklyn       Bronx            58.124000      19.600000
               Brooklyn         11.933857       2.523714
               Manhattan        25.096567       6.950597
               Queens           34.842692      10.874615
Manhattan      Bronx            24.472222       7.058889
               Brooklyn         24.551325       6.689735
               Manhattan         9.733270       1.857643
               Queens           34.581358      10.987099
               Staten Island    44.500000      15.855000
Queens         Bronx            45.772727      16.636364
               Brooklyn         37.018871      11.884194
               Manhattan        36.856233      11.943184
               Queens           12.900439       3.157485

### Übungsaufgabe

Lade den Pinguine-Datensatz aus Seaborn. Beantworte folgende Fragen:
* Was ist das durchschnittliche Gewicht eines Vertreters einer Art?
* Welche Biomasse (Summe) entfällt auf die Geschlechter im Datensatz?
* Was ist die maximale gemessene Flügellänge sowie Schnabellänge und Schnabelbreite pro Insel?
* Wie sieht die durchschnittliche Masse der jeweiligen Arten in Abhängigkeit vom Geschlecht aus? 
Gibt es da eine Gesetzmäßigkeit?
* Was sind die maximale Körpermasse, die minimale Schnabellänge sowie die durchschnittliche Flügellänge
in Abhängigkeit vom Geschlecht? Benenne die entstehende Spalten um, damit klar ist, was in welcher Spalte steht!

In [72]:
sns.get_dataset_names()
pengu_df = sns.load_dataset('penguins')
pengu_df.groupby('species')\
    .agg({'body_mass_g': 'mean'})

,body_mass_g
species,
Adelie,3700.662252
Chinstrap,3733.088235
Gentoo,5076.016260
